In [1]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import random
import os

# Setup

In [2]:
# Only change folder_path
folder_path = r"C:\Users\Nikolai\Downloads\archive"

symbol_path = os.path.join(folder_path, "symbols_valid_meta.csv")
stocks_folder = os.path.join(folder_path, "stocks")
etfs_folder = os.path.join(folder_path, "etfs")

# Load data
symbols = pd.read_csv(symbol_path)

# More info about the columns- https://www.nasdaqtrader.com/trader.aspx?id=symboldirdefs
symbols.head(5)

,Nasdaq Traded,Symbol,Security Name,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,CQS Symbol,NASDAQ Symbol,NextShares
0,Y,A,"Agilent Technologies, Inc. Common Stock",N,,N,100.0,N,NaN,A,A,N
1,Y,AA,Alcoa Corporation Common Stock,N,,N,100.0,N,NaN,AA,AA,N
2,Y,AAAU,Perth Mint Physical Gold ETF,P,,Y,100.0,N,NaN,AAAU,AAAU,N
3,Y,AACG,ATA Creativity Global - American Depositary Sh...,Q,G,N,100.0,N,N,NaN,AACG,N
4,Y,AADR,AdvisorShares Dorsey Wright ADR ETF,P,,Y,100.0,N,NaN,AADR,AADR,N


In [3]:
query = symbols["ETF"] == "Y"

# Get only Exchange-Traded Funds
etfs = symbols.loc[query].reset_index(drop=True).copy()
# Get only Stocks
stocks = symbols.loc[~query].reset_index(drop=True).copy()

In [4]:
# Stocks that we will work with are here
# NOTE: "Common Stock" means that a stock can be bought or sold by investors or traders
my_securities = {
    "MSFT": "Microsoft Corporation - Common Stock",
    "AAPL": "Apple Inc. - Common Stock",
    "NVDA": "NVIDIA Corporation - Common Stock",
    "TSLA": "Tesla, Inc. - Common Stock",
    "AMZN": "Amazon.com, Inc. - Common Stock",
    "NFLX": "Netflix, Inc. - Common Stock"
}

In [40]:
stock_symbols = list(my_securities.keys())
# Randomly pick a symbol
demo = random.choice(stock_symbols)

# NOTE: the files' names are based on the symbol name
file_path = os.path.join(stocks_folder, f"{demo}.csv")
data = pd.read_csv(file_path)
data["symbol"] = demo

# Change data type
data["Date"] = pd.to_datetime(data["Date"])

print(f"Loaded data for {demo}")
# Overall information about the dataset
data.info()

Loaded data for AAPL
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9909 entries, 0 to 9908
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       9909 non-null   datetime64[ns]
 1   Open       9909 non-null   float64       
 2   High       9909 non-null   float64       
 3   Low        9909 non-null   float64       
 4   Close      9909 non-null   float64       
 5   Adj Close  9909 non-null   float64       
 6   Volume     9909 non-null   int64         
 7   symbol     9909 non-null   object        
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 619.4+ KB


# Statistics

In [41]:
stats = data[["Date", "symbol", "Adj Close"]].copy()
stats.tail()

,Date,symbol,Adj Close
9904,2020-03-26,AAPL,258.440002
9905,2020-03-27,AAPL,247.740005
9906,2020-03-30,AAPL,254.809998
9907,2020-03-31,AAPL,254.289993
9908,2020-04-01,AAPL,240.910004


In [42]:
# Calculate period returns
# Get the percent change (decimals) on a daily basis
stats["1day"] = stats["Adj Close"].pct_change().fillna(0)

# Visualization

In [23]:
candlestick = go.Candlestick(x=data["Date"], open=data["Open"], high=data["High"], low=data["Low"], close=data["Close"])
fig = go.Figure(data = [candlestick])

# Update the plot layout
fig.update_layout(title=my_securities[demo], yaxis_title="Price", hovermode = "x unified")
fig.show()

In [ ]:
# https://plotly.com/python/candlestick-charts/
# Candlesticks
